In [4]:
import os
import sys
 
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [6]:
HeartFail = spark.read.csv('file:////home/dineshpsd67163/cloudxlab_jupyter_notebooks/Project Works/heart failier.csv',
                       header=True,inferSchema=True)

NameError: name 'spark' is not defined

In [7]:
HeartFail.show(10)

NameError: name 'HeartFail' is not defined

In [199]:
HeartFail.columns
HeartFail.select('age').show()

+----+
| age|
+----+
|75.0|
|55.0|
|65.0|
|50.0|
|65.0|
|90.0|
|75.0|
|60.0|
|65.0|
|80.0|
|75.0|
|62.0|
|45.0|
|50.0|
|49.0|
|82.0|
|87.0|
|45.0|
|70.0|
|48.0|
+----+
only showing top 20 rows



In [200]:
for col in HeartFail.columns:
    print("no. of cells in column", col, "with null values:", HeartFail.filter(HeartFail[col].isNull()).count())

no. of cells in column anaemia with null values: 0
no. of cells in column diabetes with null values: 0
no. of cells in column highBP with null values: 0
no. of cells in column sex with null values: 0
no. of cells in column smoking with null values: 0
no. of cells in column age with null values: 0
no. of cells in column death_event with null values: 0


In [201]:
HeartFail.printSchema()

root
 |-- anaemia: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- highBP: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- smoking: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- death_event: integer (nullable = true)



In [202]:
# descriptive

HeartFail.describe().show()

+-------+-------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+
|summary|            anaemia|          diabetes|            highBP|                sex|           smoking|               age|        death_event|
+-------+-------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+
|  count|               1196|              1196|              1196|               1196|              1196|              1196|               1196|
|   mean|  0.431438127090301|0.4180602006688963|0.3511705685618729| 0.6488294314381271|0.3210702341137124|60.833892976588636| 0.3210702341137124|
| stddev|0.49548414801930274|0.4934465075234322|0.4775358307256004|0.47753583072560085|0.4670830251520782|11.879868968829989|0.46708302515207845|
|    min|                  0|                 0|                 0|                  0|                 0|              40.0

In [203]:
# vector assambly 
from pyspark.ml.feature import VectorAssembler ,StandardScaler
assembler = VectorAssembler(inputCols=["anaemia","diabetes","highBP","sex","smoking","age"], 
                            outputCol="features")
Heart_feature_vec=assembler.transform(HeartFail)


In [204]:
# Split the data into train and test sets

train_data, test_data = Heart_feature_vec.randomSplit([.8,.2],seed=1234) 

In [205]:
# scalling the variables

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)
scalerModel = scaler.fit(train_data)
scaledData = scalerModel.transform(train_data)
scaledData_test = scalerModel.transform(test_data)
scaledData.select("death_event","scaledFeatures").take(1)

[Row(death_event=0, scaledFeatures=DenseVector([-0.8753, -0.8399, -0.739, -1.3547, -0.7036, -1.5902]))]

# LogisticRegression

In [206]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="death_event", featuresCol="scaledFeatures", 
                        maxIter=100, regParam=0.0001, family="multinomial")

# Train model with Training Data
lrModel = lr.fit(scaledData)
predictions = lrModel.transform(scaledData_test)
predictions.printSchema()

root
 |-- anaemia: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- highBP: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- smoking: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- death_event: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [207]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="death_event", predictionCol="prediction", metricName="accuracy")

In [208]:
#acuracy score
accuracy = evaluator.evaluate(predictions)
print(accuracy)

0.6699507389162561


In [209]:
# recall score

evaluator.setMetricName("weightedRecall")
evaluator.evaluate(predictions)

0.6699507389162562

In [210]:
# f1 score
evaluator.setMetricName("f1")
evaluator.evaluate(predictions)

0.5943000854869824

In [211]:
#precesion score
evaluator.setMetricName("weightedPrecision")
evaluator.evaluate(predictions)

0.6394967861752852

In [212]:
predictions.createOrReplaceTempView('predictions')
spark.sql("select death_event, prediction, count(*) as count \
          from predictions \
          group by death_event,prediction \
          order by prediction,death_event").show()

+-----------+----------+-----+
|death_event|prediction|count|
+-----------+----------+-----+
|          0|       0.0|  127|
|          1|       0.0|   60|
|          0|       1.0|    7|
|          1|       1.0|    9|
+-----------+----------+-----+



# Gridsearch for logistic regression:

In [213]:

from pyspark.ml.tuning import ParamGridBuilder
lr = LogisticRegression(labelCol="death_event", featuresCol="scaledFeatures")
evaluator = MulticlassClassificationEvaluator(
    labelCol="death_event", predictionCol="predictions", metricName="accuracy")
paramgrid = ParamGridBuilder() \
    .baseOn({lr.labelCol: 'death_event'}) \
    .baseOn([lr.predictionCol, 'predictions']) \
    .addGrid(lr.regParam, [1.0, 2.0]) \
    .addGrid(lr.maxIter, [1, 5]) \
    .build()

In [214]:
# K-fold
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=lr,estimatorParamMaps=paramgrid,evaluator=evaluator,numFolds=3)
    
cvModel = cv.fit(scaledData)

In [215]:
# Accuracy score
predicted_df = cvModel.transform (scaledData_test)
evaluator.evaluate(predicted_df)

0.6600985221674877

In [216]:
#precesion score
evaluator.setMetricName("weightedPrecision")
evaluator.evaluate(predicted_df)

0.43573005896770123

In [217]:
# recall score
evaluator.setMetricName("weightedRecall")
evaluator.evaluate(predicted_df)

0.6600985221674877

In [218]:
# f1 score
evaluator.setMetricName("f1")
evaluator.evaluate(predicted_df)

0.5249448188156876

In [219]:
predicted_df.createOrReplaceTempView('predicted_df')
spark.sql("select death_event, predictions, count(*) as count \
          from predicted_df \
          group by death_event,predictions \
          order by predictions,death_event").show()

+-----------+-----------+-----+
|death_event|predictions|count|
+-----------+-----------+-----+
|          0|        0.0|  134|
|          1|        0.0|   69|
+-----------+-----------+-----+



 # Random Forest

In [220]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="death_event", featuresCol="scaledFeatures", 
                            numTrees=10)
rfModel = rf.fit(scaledData)
predictions_rf = rfModel.transform(scaledData_test)

In [221]:
# Accuracy score
evaluator = MulticlassClassificationEvaluator(  
    labelCol="death_event", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_rf)
print(accuracy)

0.7241379310344828


In [222]:
# Percision score
evaluator.setMetricName("weightedPrecision")
evaluator.evaluate(predictions_rf)

0.7390804597701149

In [223]:
# Recall score
evaluator.setMetricName("weightedRecall")
evaluator.evaluate(predictions_rf)

0.7241379310344828

In [224]:
# F1 score
evaluator.setMetricName("f1")
evaluator.evaluate(predictions_rf)

0.6753788710740172

# confession matrix before grid search

In [225]:
predictions_rf.select('death_event','prediction').groupby('death_event','prediction').count().show()

+-----------+----------+-----+
|death_event|prediction|count|
+-----------+----------+-----+
|          1|       0.0|   51|
|          0|       0.0|  129|
|          1|       1.0|   18|
|          0|       1.0|    5|
+-----------+----------+-----+



In [226]:
# Create ParamGrid for Cross Validation
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

rf = RandomForestClassifier(labelCol="death_event", featuresCol="scaledFeatures")

paramGrid = (ParamGridBuilder()\
             .addGrid(rf.maxDepth, [2, 6])\
             .addGrid(rf.numTrees, [5, 20])\
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

cvModel = cv.fit(scaledData)

In [227]:
#  accuracy score 
predictions = cvModel.transform(scaledData)
evaluator.evaluate(predictions)

0.7805469749533165

In [228]:
# precision score
evaluator.setMetricName("weightedPrecision")
evaluator.evaluate(predictions)

0.8046924190934404

In [229]:
#Recall score
evaluator.setMetricName("weightedRecall")
evaluator.evaluate(predictions)

0.7995971802618328

In [230]:
# F1 score
evaluator.setMetricName("f1")
evaluator.evaluate(predictions)

0.7805469749533165

# confession matrix After grid search

In [231]:
predictions_rf.createOrReplaceTempView('rf_predictions')
spark.sql("select death_event, prediction, count(*) as count \
          from rf_predictions \
          group by death_event,prediction \
          order by death_event,prediction").show()

+-----------+----------+-----+
|death_event|prediction|count|
+-----------+----------+-----+
|          0|       0.0|  129|
|          0|       1.0|    5|
|          1|       0.0|   51|
|          1|       1.0|   18|
+-----------+----------+-----+



In [232]:
cvModel.bestModel._java_obj.getMaxDepth()

6

In [233]:
cvModel.bestModel._java_obj.getNumTrees()

5

In [234]:
spark.stop()